In [1]:
import numpy as np
import pandas as pd
import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [29]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

In [30]:
query = \
"""
select * from icu_features;
"""
res = pd.read_sql_query(query, con)
res.head()

,icustay_id,hadm_id,subject_id,age,gender,height,weight,bmi,ethnicity,insurance,...,ea_tv_pulm_htn,ea_tv_tr,ea_lv_cavity,ea_lv_diastolic,ea_lv_systolic,ea_lv_wall,ea_rv_cavity,ea_rv_diastolic_fluid,ea_rv_systolic,ea_rv_wall
0,287547,189332,109,9146 days 19:49:27,F,152.4,19.368394,8.339186,BLACK/AFRICAN AMERICAN,Medicaid,...,2.0,0.0,0.0,NaN,0.0,3.0,0.0,NaN,0.0,1.0
1,254245,192123,111,24368 days 06:56:33,F,NaN,57.560001,NaN,WHITE,Medicare,...,NaN,NaN,NaN,NaN,1.0,NaN,0.0,NaN,0.0,NaN
2,224803,130684,225,11035 days 00:58:13,M,NaN,69.800003,NaN,WHITE,Medicaid,...,NaN,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,215044,116935,249,27285 days 20:06:02,F,NaN,72.000000,NaN,WHITE,Medicare,...,1.0,1.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,257572,111199,281,21906 days 04:45:22,F,NaN,83.650002,NaN,BLACK/AFRICAN AMERICAN,Government,...,2.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN


Check that icustay_id is unique.

In [17]:
len(res['icustay_id'].unique()) == res.shape[0]

True

Total number of icustay_ids

In [26]:
res.shape[0]

61532

+ had sepsis

In [22]:
res['filter_angus_sepsis'].value_counts()

0    44112
1    17420
Name: filter_angus_sepsis, dtype: int64

+ on vasopressors

In [20]:
res['filter_vaso'].value_counts()

False    47716
True     13816
Name: filter_vaso, dtype: int64

+ first care unit or last care unit was micu

In [31]:
res['filter_micu'].value_counts()

False    39214
True     22318
Name: filter_micu, dtype: int64

+ had sepsis
+ (and/or) was on vasopressors

In [27]:
res.loc[res['filter_vaso'] & res['filter_angus_sepsis']].shape[0]

6182

In [28]:
res.loc[res['filter_vaso'] | res['filter_angus_sepsis']].shape[0]

25054

+ had sepsis or was on vasopressors
+ first or last icu = micu

In [32]:
res.loc[(res['filter_vaso'] | res['filter_angus_sepsis']) & res['filter_micu']].shape[0]

11011

+ had sepsis or was on vasopressors
+ first or last icu = micu
+ no cardiogenic shock

In [47]:
res.loc[(res['filter_vaso'] | res['filter_angus_sepsis']) 
        & res['filter_micu']
        & ~res['filter_hard_cardiogenic']
].shape[0]

10698

+ had sepsis or was on vasopressors
+ first or last icu = micu
+ no cardiogenic shock
+ not on chronic dialysis
+ was adult

In [39]:
res_ = res.loc[(res['filter_vaso'] | res['filter_angus_sepsis']) 
        & res['filter_micu']
        & ~res['filter_hard_cardiogenic']
        & ~res['filter_chronic_dialysis']
        & res['filter_adult']
]
res_.shape[0]

9681

In [41]:
res_['filter_echo'].value_counts()

False    5711
True     3970
Name: filter_echo, dtype: int64

Count outcomes based on whether or not the patient had an echo

In [50]:
res['mortality_disch'] = res['dod'] < res['dischtime']
res.groupby(['mortality_disch', 'filter_echo'])['icustay_id'].count()

mortality_disch  filter_echo
False            False          37884
                 True           17047
True             False           3896
                 True            2705
Name: icustay_id, dtype: int64

In [52]:
2705/(2705 + 3896.)

0.40978639600060596

In [53]:
17047/(37884 + 17047.)

0.3103347836376545